# Daten einlesen und bereinigen

In [50]:
#benötigte Packages installieren

import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

!pip install pyproj
from pyproj import Proj, transform

import warnings
warnings.filterwarnings('ignore')

import os

!pip install geojson
import geojson

!pip install shapely
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

# 1. Vorbereitung VBZ Daten

## 1a VBZ Fahrgastzahlen einlesen, gruppieren und mergen

In [51]:
# working directory anpassen
print(os.getcwd())
user = '/Users/' + os.getlogin()
os.chdir(user + '/Dropbox/Projekt Scripting/01 data')
print(os.getcwd())

# CSV einlesen
df_fahrgastzahlen = pd.read_csv('./Fahrgastzahlen_2021/REISENDE.csv',sep=';')
df_haltestellen = pd.read_csv('./Fahrgastzahlen_2021/HALTESTELLEN.csv',sep=';')
df_linie = pd.read_csv('./Fahrgastzahlen_2021/LINIE.csv',sep=';')
df_gefgr = pd.read_csv('./Fahrgastzahlen_2021/GEFAESSGROESSE.csv',sep=';')


C:\Users\admin\Dropbox\Projekt Scripting\01 data
C:\Users\admin\Dropbox\Projekt Scripting\01 data


In [52]:
df_fahrgastzahlen.head(3)

,Tagtyp_Id,Linien_Id,Linienname,Plan_Fahrt_Id,Richtung,Sequenz,Haltestellen_Id,Nach_Hst_Id,FZ_AB,Anzahl_Messungen,Einsteiger,Aussteiger,Besetzung,Distanz,Tage_DTV,Tage_DWV,Tage_SA,Tage_SO,Nachtnetz,Tage_SA_N,Tage_SO_N,ID_Abschnitt
0,14,9,11,88877,1,2,154,153.0,19:14:00,18,3.27778,0.22222,5.50000,399,175.27317,175.27317,0.0,0.0,0,0,0,15400153
1,14,9,11,88877,1,3,153,152.0,19:14:54,18,2.77778,1.05556,7.22222,412,175.27317,175.27317,0.0,0.0,0,0,0,15300152
2,14,9,11,88877,1,4,152,151.0,19:16:12,18,1.72222,0.22222,8.72222,383,175.27317,175.27317,0.0,0.0,0,0,0,15200151


In [53]:
#schauen, welche Infos immer gleich sind pro Haltestelle_id
df_fahrgastzahlen.loc[df_fahrgastzahlen['Haltestellen_Id']==154]

,Tagtyp_Id,Linien_Id,Linienname,Plan_Fahrt_Id,Richtung,Sequenz,Haltestellen_Id,Nach_Hst_Id,FZ_AB,Anzahl_Messungen,Einsteiger,Aussteiger,Besetzung,Distanz,Tage_DTV,Tage_DWV,Tage_SA,Tage_SO,Nachtnetz,Tage_SA_N,Tage_SO_N,ID_Abschnitt
0,14,9,11,88877,1,2,154,153.0,19:14:00,18,3.27778,0.22222,5.50000,399,175.27317,175.27317,0.0,0.00000,0,0,0,15400153
1906,14,9,11,87690,1,2,154,153.0,20:28:54,22,2.27273,0.18182,6.18182,399,175.27317,175.27317,0.0,0.00000,0,0,0,15400153
2422,14,9,11,88843,1,2,154,153.0,20:38:54,70,2.48571,0.11429,5.07143,399,175.27317,175.27317,0.0,0.00000,0,0,0,15400153
2992,4,9,11,1158,2,33,154,155.0,11:37:54,0,0.33333,6.50000,7.50000,613,0.00000,0.00000,0.0,0.00000,0,0,0,15400155
3129,4,9,11,4972,2,33,154,155.0,11:45:24,0,0.38889,6.83333,7.33333,613,0.00000,0.00000,0.0,0.00000,0,0,0,15400155
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1208643,4,81,12,59484,2,10,154,153.0,18:46:36,13,1.84615,4.61538,21.15385,504,52.00000,0.00000,52.0,0.00000,0,0,0,15400153
1208661,4,81,12,68423,2,10,154,153.0,19:01:36,8,0.12500,1.75000,16.62500,504,52.00000,0.00000,52.0,0.00000,0,0,0,15400153
1208679,4,81,12,45198,2,10,154,153.0,19:16:36,5,0.40000,1.80000,22.80000,504,52.00000,0.00000,52.0,0.00000,0,0,0,15400153
1208925,3,9,11,7887,1,2,154,153.0,20:38:54,0,0.37500,0.00000,0.75000,399,11.18033,0.00000,0.0,11.18033,0,0,0,15400153


In [54]:
#traffic_total per Haltestelle berechnen
df_fahrgastzahlen['traffic_total'] = df_fahrgastzahlen['Einsteiger'] + df_fahrgastzahlen['Aussteiger']

In [55]:
#Schauen, wie viele Messungen durchgeführt wurden
df_fahrgastzahlen['Anzahl_Messungen'].value_counts()

1      107896
0       99782
3       82006
4       79338
5       79289
        ...  
115         4
118         4
174         4
120         4
117         4
Name: Anzahl_Messungen, Length: 163, dtype: int64

In [56]:
#Diejenigen Daten rausfiltern, wo es keine Messung gab
filt=df_fahrgastzahlen['Anzahl_Messungen']>0
df_fahrgastzahlen=df_fahrgastzahlen[filt]

In [57]:
df_haltestellen.head(3)

,Haltestellen_Id,Haltestellennummer,Haltestellenkurzname,Haltestellenlangname
0,509,6594,AHOR,"Adliswil, Ahornweg"
1,511,148,ABAD01,"Adliswil, Badstrasse"
2,513,28,BADL01,"Adliswil, Bahnhof"


In [58]:
df_linie.head(3)

,Linien_Id,Linienname,VSYS,Linienname_Fahrgastauskunft
0,31,10,T,10
1,9,11,T,11
2,81,12,T,12


In [59]:
#Fahrgastzahlen nochmals um zu schauen, welche Variable wie aggregiert werden muss
df_fahrgastzahlen.head(1)

,Tagtyp_Id,Linien_Id,Linienname,Plan_Fahrt_Id,Richtung,Sequenz,Haltestellen_Id,Nach_Hst_Id,FZ_AB,Anzahl_Messungen,Einsteiger,Aussteiger,Besetzung,Distanz,Tage_DTV,Tage_DWV,Tage_SA,Tage_SO,Nachtnetz,Tage_SA_N,Tage_SO_N,ID_Abschnitt,traffic_total
0,14,9,11,88877,1,2,154,153.0,19:14:00,18,3.27778,0.22222,5.5,399,175.27317,175.27317,0.0,0.0,0,0,0,15400153,3.5


In [60]:
## Fahrgastzahlen gruppieren und je nach Variable unterschiedlich aggregieren

df_fahrgastzahlen_gp = (
df_fahrgastzahlen.groupby(['Haltestellen_Id','Linien_Id'])
                .agg(Linienname=('Linienname', 'first')
                    ,Plan_Fahrt_Id=('Plan_Fahrt_Id','first') 
                    ,Anzahl_Messungen=('Anzahl_Messungen', sum)
                    ,Einsteiger=('Einsteiger', np.mean)
                    ,Aussteiger=('Aussteiger', np.mean)
                    ,Besetzung=('Besetzung', np.mean)
                    ,traffic_total=('traffic_total', np.mean)
                    ,Distanz_m=('Distanz', np.mean)
                    )
                        ).reset_index()

In [61]:
df_fahrgastzahlen_gp.head(100)

,Haltestellen_Id,Linien_Id,Linienname,Plan_Fahrt_Id,Anzahl_Messungen,Einsteiger,Aussteiger,Besetzung,traffic_total,Distanz_m
0,4,3,916,102001,5309,2.260442,1.916962,10.838398,4.177404,606.862245
1,4,7,33,68716,21038,1.515205,1.302838,3.030410,2.818044,192.500000
2,4,11,4,88973,30122,3.012010,2.403962,6.020661,5.415972,185.103179
3,4,17,2,71915,13063,2.269434,2.670671,4.538867,4.940104,185.000000
4,4,34,912,62992,4922,2.973354,2.584563,15.311116,5.557917,604.570048
5,4,35,910,44534,6832,4.469661,3.321159,8.939323,7.790820,547.500000
6,4,94,348,114020,81,1.611168,0.787415,8.294501,2.398582,498.714286
7,5,3,916,102001,5309,0.238072,0.225833,12.181104,0.463906,525.469388
8,5,34,912,62992,4922,0.229136,0.221816,17.298230,0.450951,524.111111
9,5,35,910,44567,3443,0.101601,0.076213,6.642318,0.177814,556.000000


In [62]:
df_fahrgastzahlen_gp[df_fahrgastzahlen_gp.Haltestellen_Id == 4]

,Haltestellen_Id,Linien_Id,Linienname,Plan_Fahrt_Id,Anzahl_Messungen,Einsteiger,Aussteiger,Besetzung,traffic_total,Distanz_m
0,4,3,916,102001,5309,2.260442,1.916962,10.838398,4.177404,606.862245
1,4,7,33,68716,21038,1.515205,1.302838,3.030410,2.818044,192.500000
2,4,11,4,88973,30122,3.012010,2.403962,6.020661,5.415972,185.103179
3,4,17,2,71915,13063,2.269434,2.670671,4.538867,4.940104,185.000000
4,4,34,912,62992,4922,2.973354,2.584563,15.311116,5.557917,604.570048
5,4,35,910,44534,6832,4.469661,3.321159,8.939323,7.790820,547.500000
6,4,94,348,114020,81,1.611168,0.787415,8.294501,2.398582,498.714286


In [63]:
df_fahrgastzahlen_gp.Haltestellen_Id.value_counts().to_frame()

# Hier sieht man, wie viele Linien pro Haltestelle befahren werden

,Haltestellen_Id
23,25
78,17
94,17
68,13
129,12
...,...
566,1
91,1
568,1
569,1


In [64]:
df_fahrgastzahlen_gp.sort_values('Anzahl_Messungen')
print('Anzahl Zeilen Total:')
print (df_fahrgastzahlen_gp.shape[0])
print ('Anzahl Zeilen mit < 20 Messungen:')
print (df_fahrgastzahlen_gp.loc[df_fahrgastzahlen_gp.Anzahl_Messungen<20].shape[0])

Anzahl Zeilen Total:
2127
Anzahl Zeilen mit < 20 Messungen:
67


In [65]:
# Dataframes zusammenführen
df_fahrgastzahlen_merge = (df_fahrgastzahlen_gp.merge(df_haltestellen,on='Haltestellen_Id',how='inner')
                           .merge(df_linie,on=('Linien_Id','Linienname'), how='inner')
                          )
df_fahrgastzahlen_merge

,Haltestellen_Id,Linien_Id,Linienname,Plan_Fahrt_Id,Anzahl_Messungen,Einsteiger,Aussteiger,Besetzung,traffic_total,Distanz_m,Haltestellennummer,Haltestellenkurzname,Haltestellenlangname,VSYS,Linienname_Fahrgastauskunft
0,4,3,916,102001,5309,2.260442,1.916962,10.838398,4.177404,606.862245,3041,BTIE,"Zürich, Bahnhof Tiefenbrunnen",BP,916
1,5,3,916,102001,5309,0.238072,0.225833,12.181104,0.463906,525.469388,418,BUEB,"Zollikon, Bahnübergang",BP,916
2,6,3,916,102001,5309,0.708526,0.750994,11.368270,1.459520,397.579082,824,FBST,"Zollikon, Felbenstrasse",BP,916
3,7,3,916,102001,5309,1.305883,1.134898,10.000685,2.440781,442.272959,715,DUFP,"Zollikon, Dufourplatz",BP,916
4,8,3,916,102001,5309,1.191597,1.329759,7.951429,2.521356,423.140306,476,BEUG,"Zollikon, Beugi",BP,916
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2122,590,58,325,40536,3149,0.280595,0.504061,0.561190,0.784657,176.500000,6698,DWEI,"Dietikon, Weinberg",BL,325
2123,545,83,753,69316,18,2.666670,0.000000,2.666670,2.666670,3131.000000,1311,IKLO,"Zürich, Im Klösterli",BG,E
2124,594,83,753,69317,18,2.611110,0.111110,26.111110,2.722220,2688.000000,6214,URSP,"Gockhausen, Ursprungstrasse",BG,E
2125,595,83,753,69316,36,11.472220,1.000000,23.611110,12.472220,197.500000,1006,GDOR,"Gockhausen, Dorf",BG,E


In [66]:
#neue Variablen bilden
df_fahrgastzahlen_merge['halt_lin_kurz']=df_fahrgastzahlen_merge['Haltestellen_Id'].astype(str)+'-'+df_fahrgastzahlen_merge['Linienname_Fahrgastauskunft'].astype(str)
df_fahrgastzahlen_merge['halt_lin_lang']=df_fahrgastzahlen_merge['Haltestellenlangname'].astype(str)+' Linie '+df_fahrgastzahlen_merge['Linienname_Fahrgastauskunft'].astype(str)
df_fahrgastzahlen_merge.head(3)

,Haltestellen_Id,Linien_Id,Linienname,Plan_Fahrt_Id,Anzahl_Messungen,Einsteiger,Aussteiger,Besetzung,traffic_total,Distanz_m,Haltestellennummer,Haltestellenkurzname,Haltestellenlangname,VSYS,Linienname_Fahrgastauskunft,halt_lin_kurz,halt_lin_lang
0,4,3,916,102001,5309,2.260442,1.916962,10.838398,4.177404,606.862245,3041,BTIE,"Zürich, Bahnhof Tiefenbrunnen",BP,916,4-916,"Zürich, Bahnhof Tiefenbrunnen Linie 916"
1,5,3,916,102001,5309,0.238072,0.225833,12.181104,0.463906,525.469388,418,BUEB,"Zollikon, Bahnübergang",BP,916,5-916,"Zollikon, Bahnübergang Linie 916"
2,6,3,916,102001,5309,0.708526,0.750994,11.368270,1.459520,397.579082,824,FBST,"Zollikon, Felbenstrasse",BP,916,6-916,"Zollikon, Felbenstrasse Linie 916"


In [67]:
# schauen ob es sich um Nachtnetz oder Sonderlinien handelt und diese dann rausfiltern
# Für Sonderlinie das letzte Zeichen extrahieren
df_fahrgastzahlen_merge['Linientyp_S'] = df_fahrgastzahlen_merge['Linienname_Fahrgastauskunft'].str[-1:]
# Für Nachtlinie die ersten beiden Zeichen extrahieren
df_fahrgastzahlen_merge['Linientyp_N'] = df_fahrgastzahlen_merge['Linienname_Fahrgastauskunft'].str[0:2]

df_fahrgastzahlen_merge.head(1)

,Haltestellen_Id,Linien_Id,Linienname,Plan_Fahrt_Id,Anzahl_Messungen,Einsteiger,Aussteiger,Besetzung,traffic_total,Distanz_m,Haltestellennummer,Haltestellenkurzname,Haltestellenlangname,VSYS,Linienname_Fahrgastauskunft,halt_lin_kurz,halt_lin_lang,Linientyp_S,Linientyp_N
0,4,3,916,102001,5309,2.260442,1.916962,10.838398,4.177404,606.862245,3041,BTIE,"Zürich, Bahnhof Tiefenbrunnen",BP,916,4-916,"Zürich, Bahnhof Tiefenbrunnen Linie 916",6,91


In [68]:
df_fahrgastzahlen_merge.loc[df_fahrgastzahlen_merge['Linientyp_S'] == 'S', 'linientyp_spez'] = 1
df_fahrgastzahlen_merge.loc[df_fahrgastzahlen_merge['Linientyp_N'].str.contains('N'), 'linientyp_spez'] = 1
df_fahrgastzahlen_merge.head(100)

,Haltestellen_Id,Linien_Id,Linienname,Plan_Fahrt_Id,Anzahl_Messungen,Einsteiger,Aussteiger,Besetzung,traffic_total,Distanz_m,Haltestellennummer,Haltestellenkurzname,Haltestellenlangname,VSYS,Linienname_Fahrgastauskunft,halt_lin_kurz,halt_lin_lang,Linientyp_S,Linientyp_N,linientyp_spez
0,4,3,916,102001,5309,2.260442,1.916962,10.838398,4.177404,606.862245,3041,BTIE,"Zürich, Bahnhof Tiefenbrunnen",BP,916,4-916,"Zürich, Bahnhof Tiefenbrunnen Linie 916",6,91,NaN
1,5,3,916,102001,5309,0.238072,0.225833,12.181104,0.463906,525.469388,418,BUEB,"Zollikon, Bahnübergang",BP,916,5-916,"Zollikon, Bahnübergang Linie 916",6,91,NaN
2,6,3,916,102001,5309,0.708526,0.750994,11.368270,1.459520,397.579082,824,FBST,"Zollikon, Felbenstrasse",BP,916,6-916,"Zollikon, Felbenstrasse Linie 916",6,91,NaN
3,7,3,916,102001,5309,1.305883,1.134898,10.000685,2.440781,442.272959,715,DUFP,"Zollikon, Dufourplatz",BP,916,7-916,"Zollikon, Dufourplatz Linie 916",6,91,NaN
4,8,3,916,102001,5309,1.191597,1.329759,7.951429,2.521356,423.140306,476,BEUG,"Zollikon, Beugi",BP,916,8-916,"Zollikon, Beugi Linie 916",6,91,NaN
5,9,3,916,102001,5297,0.694024,0.759047,6.400046,1.453071,325.500000,1013,GOHA,"Zollikon, Goldhaldenplatz",BP,916,9-916,"Zollikon, Goldhaldenplatz Linie 916",6,91,NaN
6,10,3,916,102001,5297,0.443287,0.506121,5.461854,0.949408,316.500000,610,BUEH,"Küsnacht ZH, Bühlstrasse",BP,916,10-916,"Küsnacht ZH, Bühlstrasse Linie 916",6,91,NaN
7,11,3,916,102124,5297,0.278060,0.378746,4.870674,0.656807,385.000000,1011,GOBA,"Küsnacht ZH, Goldbacherstrasse",BP,916,11-916,"Küsnacht ZH, Goldbacherstrasse Linie 916",6,91,NaN
8,12,3,916,102124,5297,0.325260,0.445205,4.348580,0.770465,403.238462,516,BOGL,"Küsnacht ZH, Boglerenstrasse",BP,916,12-916,"Küsnacht ZH, Boglerenstrasse Linie 916",6,91,NaN
9,13,3,916,102124,5340,0.814023,0.975974,3.986891,1.789997,309.864796,2168,SIHE,"Küsnacht ZH, Schiedhaldenstr.",BP,916,13-916,"Küsnacht ZH, Schiedhaldenstr. Linie 916",6,91,NaN


In [69]:
# Speziallinien rausfiltern
filt= df_fahrgastzahlen_merge['linientyp_spez']!=1
df_fahrgastzahlen_merge=df_fahrgastzahlen_merge[filt]

In [70]:
#0 total_traffic rausfiltern
filt2= df_fahrgastzahlen_merge['traffic_total']!=0
df_fahrgastzahlen_merge=df_fahrgastzahlen_merge[filt2]

## 1b Haltestellen Geodaten einlesen und mit Fahrgastzahlen mergen

In [71]:
#Daten einlesen
df_haltestellen_geo = pd.read_csv('./Haltestellen_des_offentlichen_Verkehrs_-OGD/ZVV_HALTESTELLEN_P.csv')
df_haltestellen_geo.head(3)

,NHSTID,DIVA_NR,Y_KOORD,X_KOORD,CHSTNAME,CNAME,VTYP,HSTTYP,HAFAS_PRIO,LINKTEXT,ZONEN,SYMB_CODE,SYMB_TEXT,LINIEN,E,N
0,8583069,6796,NaN,NaN,"Ottenbach, Rigiblick",Ottenbach,Bus,&nbsp;,0,NaN,156.0,2,Bus,212,2673133,1236781
1,8590618,2982,NaN,NaN,"Geroldswil, Zentrum",Geroldswil,Bus,&nbsp;,0,NaN,154.0,2,Bus,"301,302,304,N34",2673255,1252747
2,8572606,2733,NaN,NaN,"Ottenbach, Ulmenhof",Ottenbach,Bus,&nbsp;,0,NaN,156.0,2,Bus,"215,N22",2673318,1236747


In [72]:
#Daten mergen
df_fahrgastzahlen_geo=df_fahrgastzahlen_merge.merge(df_haltestellen_geo,left_on='Haltestellennummer',right_on='DIVA_NR', how='left')

In [73]:
df_fahrgastzahlen_geo.head(3)

,Haltestellen_Id,Linien_Id,Linienname,Plan_Fahrt_Id,Anzahl_Messungen,Einsteiger,Aussteiger,Besetzung,traffic_total,Distanz_m,Haltestellennummer,Haltestellenkurzname,Haltestellenlangname,VSYS,Linienname_Fahrgastauskunft,halt_lin_kurz,halt_lin_lang,Linientyp_S,Linientyp_N,linientyp_spez,NHSTID,DIVA_NR,Y_KOORD,X_KOORD,CHSTNAME,CNAME,VTYP,HSTTYP,HAFAS_PRIO,LINKTEXT,ZONEN,SYMB_CODE,SYMB_TEXT,LINIEN,E,N
0,4,3,916,102001,5309,2.260442,1.916962,10.838398,4.177404,606.862245,3041,BTIE,"Zürich, Bahnhof Tiefenbrunnen",BP,916,4-916,"Zürich, Bahnhof Tiefenbrunnen Linie 916",6,91,NaN,8576182,3041,NaN,NaN,"Zürich, Bahnhof Tiefenbrunnen",Zürich,"Bus,Tram,Trolleybus",&nbsp;,2,NaN,110.0,3,Bus Tram,"2,4,33,910,912,916,N91",2684803,1245002
1,5,3,916,102001,5309,0.238072,0.225833,12.181104,0.463906,525.469388,418,BUEB,"Zollikon, Bahnübergang",BP,916,5-916,"Zollikon, Bahnübergang Linie 916",6,91,NaN,8576201,418,NaN,NaN,"Zollikon, Bahnübergang",Zollikon,Bus,&nbsp;,0,NaN,140.0,2,Bus,"910,912,916,N91",2685196,1244508
2,6,3,916,102001,5309,0.708526,0.750994,11.368270,1.459520,397.579082,824,FBST,"Zollikon, Felbenstrasse",BP,916,6-916,"Zollikon, Felbenstrasse Linie 916",6,91,NaN,8576202,824,NaN,NaN,"Zollikon, Felbenstrasse",Zollikon,Bus,&nbsp;,0,NaN,140.0,2,Bus,"910,912,916,N91",2685467,1244210


## 1c Koordinaten in WGS84 Format umwandeln

In [74]:
# LV95 zu WGS84
inProj = Proj('epsg:2056')
outProj = Proj('epsg:4326')
x1,y1 = df_fahrgastzahlen_geo['E'], df_fahrgastzahlen_geo['N']
x2,y2 = transform(inProj, outProj, x1,y1)

In [75]:
df_fahrgastzahlen_geo['Koord_WGS84_E'] = x2
df_fahrgastzahlen_geo['Koord_WGS84_N'] = y2

In [76]:
df_fahrgastzahlen_geo.head(3)

,Haltestellen_Id,Linien_Id,Linienname,Plan_Fahrt_Id,Anzahl_Messungen,Einsteiger,Aussteiger,Besetzung,traffic_total,Distanz_m,Haltestellennummer,Haltestellenkurzname,Haltestellenlangname,VSYS,Linienname_Fahrgastauskunft,halt_lin_kurz,halt_lin_lang,Linientyp_S,Linientyp_N,linientyp_spez,NHSTID,DIVA_NR,Y_KOORD,X_KOORD,CHSTNAME,CNAME,VTYP,HSTTYP,HAFAS_PRIO,LINKTEXT,ZONEN,SYMB_CODE,SYMB_TEXT,LINIEN,E,N,Koord_WGS84_E,Koord_WGS84_N
0,4,3,916,102001,5309,2.260442,1.916962,10.838398,4.177404,606.862245,3041,BTIE,"Zürich, Bahnhof Tiefenbrunnen",BP,916,4-916,"Zürich, Bahnhof Tiefenbrunnen Linie 916",6,91,NaN,8576182,3041,NaN,NaN,"Zürich, Bahnhof Tiefenbrunnen",Zürich,"Bus,Tram,Trolleybus",&nbsp;,2,NaN,110.0,3,Bus Tram,"2,4,33,910,912,916,N91",2684803,1245002,47.350416,8.560994
1,5,3,916,102001,5309,0.238072,0.225833,12.181104,0.463906,525.469388,418,BUEB,"Zollikon, Bahnübergang",BP,916,5-916,"Zollikon, Bahnübergang Linie 916",6,91,NaN,8576201,418,NaN,NaN,"Zollikon, Bahnübergang",Zollikon,Bus,&nbsp;,0,NaN,140.0,2,Bus,"910,912,916,N91",2685196,1244508,47.345922,8.566101
2,6,3,916,102001,5309,0.708526,0.750994,11.368270,1.459520,397.579082,824,FBST,"Zollikon, Felbenstrasse",BP,916,6-916,"Zollikon, Felbenstrasse Linie 916",6,91,NaN,8576202,824,NaN,NaN,"Zollikon, Felbenstrasse",Zollikon,Bus,&nbsp;,0,NaN,140.0,2,Bus,"910,912,916,N91",2685467,1244210,47.343207,8.569630


In [77]:
#### Column names als lowercase konvertieren
df_fahrgastzahlen_geo.columns=df_fahrgastzahlen_geo.columns.str.lower()
df_fahrgastzahlen_geo.columns

Index(['haltestellen_id', 'linien_id', 'linienname', 'plan_fahrt_id',
       'anzahl_messungen', 'einsteiger', 'aussteiger', 'besetzung',
       'traffic_total', 'distanz_m', 'haltestellennummer',
       'haltestellenkurzname', 'haltestellenlangname', 'vsys',
       'linienname_fahrgastauskunft', 'halt_lin_kurz', 'halt_lin_lang',
       'linientyp_s', 'linientyp_n', 'linientyp_spez', 'nhstid', 'diva_nr',
       'y_koord', 'x_koord', 'chstname', 'cname', 'vtyp', 'hsttyp',
       'hafas_prio', 'linktext', 'zonen', 'symb_code', 'symb_text', 'linien',
       'e', 'n', 'koord_wgs84_e', 'koord_wgs84_n'],
      dtype='object')

In [78]:
#nur relevante Spalten wählen (tbd)
df_fahrgastzahlen_geo=df_fahrgastzahlen_geo[['haltestellen_id','linien_id',
'linienname','plan_fahrt_id','einsteiger','aussteiger','besetzung','traffic_total','anzahl_messungen','haltestellenlangname',
'distanz_m', 'halt_lin_kurz', 'halt_lin_lang', 'zonen', 'vsys', 'linienname_fahrgastauskunft',
'koord_wgs84_n','koord_wgs84_e']]
print(df_fahrgastzahlen_geo.shape)
df_fahrgastzahlen_geo.head(1)                              

(1631, 18)


,haltestellen_id,linien_id,linienname,plan_fahrt_id,einsteiger,aussteiger,besetzung,traffic_total,anzahl_messungen,haltestellenlangname,distanz_m,halt_lin_kurz,halt_lin_lang,zonen,vsys,linienname_fahrgastauskunft,koord_wgs84_n,koord_wgs84_e
0,4,3,916,102001,2.260442,1.916962,10.838398,4.177404,5309,"Zürich, Bahnhof Tiefenbrunnen",606.862245,4-916,"Zürich, Bahnhof Tiefenbrunnen Linie 916",110.0,BP,916,8.560994,47.350416


## 1e Quartier den Koordinaten zuteilen

In [79]:
map_path = './Geodaten/stzh.adm_statzonen_map.geojson'

In [80]:
with open(map_path) as f:
    gj = geojson.load(f)
features = gj['features']

In [81]:
#erstelle list of tuples für jede Statistische Zone
dict_of_q_tuples = {}
for y in range(0,len(features)):
    q_tuples = []
    for x in range(0,len(features[y]['geometry']['coordinates'][0])):
        q_tuples.append(tuple(features[y]['geometry']['coordinates'][0][x]))
    dict_of_q_tuples[features[y]['properties']['stzname']] = q_tuples

In [82]:
#Schau mal ob Haltestelle in Polygon von Quartier sich befindet; entsprechend zuteilen
quartier_liste = []

for i in range(0,len(df_fahrgastzahlen_geo)):
    counter = 0
    for y in range(0, len(features)):
        polygon = Polygon(dict_of_q_tuples[features[y]['properties']['stzname']])
        point = Point(df_fahrgastzahlen_geo['koord_wgs84_n'][i], df_fahrgastzahlen_geo['koord_wgs84_e'][i])
        if polygon.contains(point) or point.touches(polygon):
            quartier_liste.append(features[y]['properties']['stzname'])
            counter = 1
            break
        if counter == 1: break
    if counter == 0: quartier_liste.append('Keine Zone')

In [83]:
len(quartier_liste)

1631

In [84]:
df_fahrgastzahlen_geo['Quartier'] = quartier_liste

In [85]:
#Schau mal ohne Zone - es sind Haltestellen ausserhalb Züri-Stadt
kq_filter = df_fahrgastzahlen_geo.loc[df_fahrgastzahlen_geo['Quartier'] == 'Keine Zone',:]

## 1f Kapazität der Fahrtlinien

In [86]:
#Daten mergen
df_fahrgastzahlen_geo_gg=df_fahrgastzahlen_geo.merge(df_gefgr,left_on='plan_fahrt_id',right_on='Plan_Fahrt_Id', how='left')

In [87]:
#Belastung berechnen und nan ersetzen
#SITZPLÄTZE
df_fahrgastzahlen_geo_gg['belastung'] = df_fahrgastzahlen_geo_gg['besetzung']/df_fahrgastzahlen_geo_gg['SITZPLAETZE']
df_fahrgastzahlen_geo_gg['belastung']=df_fahrgastzahlen_geo_gg['belastung'].fillna(0).copy()

In [88]:
#Spalten löschen
df_fahrgastzahlen_geo_gg=df_fahrgastzahlen_geo_gg[['haltestellen_id','linien_id',
'linienname','plan_fahrt_id','belastung','einsteiger','aussteiger','besetzung','traffic_total','anzahl_messungen','haltestellenlangname',
'distanz_m', 'halt_lin_kurz', 'halt_lin_lang', 'zonen', 'vsys', 'linienname_fahrgastauskunft',
'koord_wgs84_n','koord_wgs84_e','Quartier']]

## Daten exportieren

In [89]:
#### Daten exportieren
df_fahrgastzahlen_geo_gg.to_csv('./df_fahrgastzahlen_geo.csv', index=False)

# 2 Geodaten Bevölkerung

## 2a Geodaten Bevölkerung Einlesen

In [90]:
# Punkt-Datensatz einlesen
#df_bev_geo_p = pd.read_csv('./Raumliche_Bevolkerungsstatistik_-OGD/BEVOELKERUNG_HA_P.csv')
df_bev_geo_p = pd.read_csv('./Geodaten/BEVOELKERUNG_HA_P.csv')
df_bev_geo_p.head(3)

,RECORDID,PERS_N,J_0_6_P,J_7_15_P,J_16_19_P,J_20_24_P,J_25_44_P,J_45_64_P,J_65_79_P,J_80PLUS_P,ALTERSQUOT,JUGENDQUOT,GESAMTQUOT,MANN_N,FRAU_N,FRAU_P,SCHWEIZ_N,AUSLAND_N,AUSLAND_P,FLAECHE_HA,DICHTE_PHA,STICHTAG,E,N
0,1277028,43,9.3,4.7,2.3,0.0,27.9,34.9,4.7,16.3,33.3,25.9,59.3,22,21,48.8,23,20,46.5,1,43,20220630,2672150,1250250
1,1275534,122,7.4,9.0,4.1,4.1,30.3,28.7,10.7,5.7,26.0,32.5,58.4,60,62,50.8,50,72,59.0,1,122,20220630,2672150,1250550
2,1272558,15,0.0,0.0,0.0,6.7,60.0,20.0,6.7,6.7,15.4,0.0,15.4,6,9,60.0,8,7,46.7,1,15,20220630,2672350,1250050


In [91]:
df_bev_geo_p.isnull().values.any()

False

Es gibt zwar keine Nullwerte, jedoch viele -999 welche ersetzt werden müssen

In [92]:
### alle -999 mit 0 ersetzen

df_bev_geo_p=df_bev_geo_p.replace(-999,0)
df_bev_geo_p.head(3)

,RECORDID,PERS_N,J_0_6_P,J_7_15_P,J_16_19_P,J_20_24_P,J_25_44_P,J_45_64_P,J_65_79_P,J_80PLUS_P,ALTERSQUOT,JUGENDQUOT,GESAMTQUOT,MANN_N,FRAU_N,FRAU_P,SCHWEIZ_N,AUSLAND_N,AUSLAND_P,FLAECHE_HA,DICHTE_PHA,STICHTAG,E,N
0,1277028,43,9.3,4.7,2.3,0.0,27.9,34.9,4.7,16.3,33.3,25.9,59.3,22,21,48.8,23,20,46.5,1,43,20220630,2672150,1250250
1,1275534,122,7.4,9.0,4.1,4.1,30.3,28.7,10.7,5.7,26.0,32.5,58.4,60,62,50.8,50,72,59.0,1,122,20220630,2672150,1250550
2,1272558,15,0.0,0.0,0.0,6.7,60.0,20.0,6.7,6.7,15.4,0.0,15.4,6,9,60.0,8,7,46.7,1,15,20220630,2672350,1250050


## 2b Koordinaten in WGS84 Format umwandeln

In [93]:
# LV95 zu WGS84
inProj = Proj('epsg:2056')
outProj = Proj('epsg:4326')
x1,y1 = df_bev_geo_p['E'], df_bev_geo_p['N']
x2,y2 = transform(inProj, outProj, x1,y1)

In [94]:
df_bev_geo_p['Koord_WGS84_E'] = x2
df_bev_geo_p['Koord_WGS84_N'] = y2

In [95]:
df_bev_geo_p.head(3)

,RECORDID,PERS_N,J_0_6_P,J_7_15_P,J_16_19_P,J_20_24_P,J_25_44_P,J_45_64_P,J_65_79_P,J_80PLUS_P,ALTERSQUOT,JUGENDQUOT,GESAMTQUOT,MANN_N,FRAU_N,FRAU_P,SCHWEIZ_N,AUSLAND_N,AUSLAND_P,FLAECHE_HA,DICHTE_PHA,STICHTAG,E,N,Koord_WGS84_E,Koord_WGS84_N
0,1277028,43,9.3,4.7,2.3,0.0,27.9,34.9,4.7,16.3,33.3,25.9,59.3,22,21,48.8,23,20,46.5,1,43,20220630,2672150,1250250,47.399120,8.394395
1,1275534,122,7.4,9.0,4.1,4.1,30.3,28.7,10.7,5.7,26.0,32.5,58.4,60,62,50.8,50,72,59.0,1,122,20220630,2672150,1250550,47.401818,8.394444
2,1272558,15,0.0,0.0,0.0,6.7,60.0,20.0,6.7,6.7,15.4,0.0,15.4,6,9,60.0,8,7,46.7,1,15,20220630,2672350,1250050,47.397299,8.397012


In [96]:
#### Column names als lowercase konvertieren
df_bev_geo_p.columns=df_bev_geo_p.columns.str.lower()

In [97]:
###auf relevante Spalten filtern
spalt_bev=['pers_n','koord_wgs84_e','koord_wgs84_n']
df_bev_geo_p=df_bev_geo_p[spalt_bev]
df_bev_geo_p.head(3)

,pers_n,koord_wgs84_e,koord_wgs84_n
0,43,47.399120,8.394395
1,122,47.401818,8.394444
2,15,47.397299,8.397012


In [98]:
#csv speichern
df_bev_geo_p.to_csv('./df_bev_geo_p.csv', index=False)